

# Micrograd

Дедлайн: 18.02. Скопируйте ноутбук себе, решите задания. Откройте ноутбук для редактирования. Отправьте ссылку на ноутбук в форму.  https://forms.gle/MWNQqFbUaZvwB5j56

---

1. watch the [micrograd video](https://www.youtube.com/watch?v=VMj-3S1tku0) on YouTube
2. come back and complete these exercises to level up :)

# Section 0: Statistics (2 pts)

Given on the lecture. See slides for more details...




Хотим сравнить значения метрик и проверить гипотезы

$ H_0 : F_1 = F_2 \text{ vs } H_1 : F_1 \neq F_2 $

Зафиксируем уровень значимость $\alpha = 0.05$.

Для проверки воспользуемся двухвыборочным критерием Стьюдента($t$-критерий).

\begin{gather*}
t = \frac{| \bar{m_1} - \bar{m_2} |}{\sqrt{\frac{\sigma_1^2}{N_1} + \frac{\sigma_2^2}{N_2}}}, где \\
\\
N_1 = 5, N_2 = 5, \bar{m_1} = 92.82, \bar{m_2} = 92.66, \sigma_1 = 0.82, \sigma_2 = 0.11
\end{gather*}

После подсчётов имеем, что $ t = 0.43 $. 

По таблице при $ DF = 2N - 2 = 8$ и $\ alpha = 0.05 $ имеем $ t_{0.05} = 2,306$.

$ t < t_{0.05} \implies $ результат не статистически значимый и мы не можем отвегнуть гипотезу $ H_0 $.

У нас нет достаточных доказательств, чтобы сказать, что средние значения метрик двух моделей различны.




***
Не понял зачем было дано какое конкретно распределение данных на классы. Я что-то не учёл?

# Section 1: derivatives

In [59]:
# here is a mathematical expression that takes 3 inputs and produces one output
from math import sin, cos

def f(a, b, c):
  return -a**3 + sin(3*b) - 1.0/c + b**2.5 - a**0.5

print(f(2, 3, 4))

6.336362190988558


In [60]:
# write the function df that returns the analytical gradient of f
# i.e. use your skills from calculus to take the derivative, then implement the formula
# if you do not calculus then feel free to ask wolframalpha, e.g.:
# https://www.wolframalpha.com/input?i=d%2Fda%28sin%283*a%29%29%29

def gradf(a, b, c):
  return [-3*a**2 - 1.0/(2*a**0.5), 3*cos(3*b) + 2.5*b**1.5, 1.0/(c**2)] # return [df/da, df/db, df/dc]

# expected answer is the list of
ans = [-12.353553390593273, 10.25699027111255, 0.0625]
yours = gradf(2, 3, 4)
for dim in range(3):
  ok = 'OK' if abs(yours[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {yours[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553390593273
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027111255
OK for dim 2: expected 0.0625, yours returns 0.0625


In [61]:
# now estimate the gradient numerically without any calculus, using
# the approximation we used in the video.
# you should not call the function df from the last cell

# -----------

def derivative_f(a, b, c):
  h = 0.000001
  df = [f(a+h,b,c) - f(a,b,c), f(a,b+h,c) - f(a,b,c), f(a,b,c+h) - f(a,b,c)]
  return [val/h for val in df]

numerical_grad = derivative_f(2, 3, 4)

# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353559348809995
OK for dim 1: expected 10.25699027111255, yours returns 10.256991666679482
OK for dim 2: expected 0.0625, yours returns 0.062499984743169534


In [62]:
# there is an alternative formula that provides a much better numerical
# approximation to the derivative of a function.
# learn about it here: https://en.wikipedia.org/wiki/Symmetric_derivative
# implement it. confirm that for the same step size h this version gives a
# better approximation.

# -----------

def symmetric_derivative_f(a, b, c):
  h = 0.000001
  df = [f(a+h,b,c) - f(a-h,b,c), f(a,b+h,c) - f(a,b-h,c), f(a,b,c+h) - f(a,b,c-h)]
  return [val/(2*h) for val in df]

numerical_grad2 = symmetric_derivative_f(2, 3, 4)

# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad2[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad2[dim]}")


for dim in range(3):
  dif = abs(numerical_grad[dim] - ans[dim]) - abs(numerical_grad2[dim] - ans[dim])
  if dif > 0:
    print(f"Better approximation for dim {dim}: accuracy improvement {dif}")
  else:
    print(f"Worse approximation for dim {dim}")

OK for dim 0: expected -12.353553390593273, yours returns -12.353553391353245
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027401572
OK for dim 2: expected 0.0625, yours returns 0.06250000028629188
Better approximation for dim 0: accuracy improvement 5.95745675013859e-06
Better approximation for dim 1: accuracy improvement 1.3926637620897964e-06
Better approximation for dim 2: accuracy improvement 1.4970538586567272e-08


# Section 2: support for softmax, relu, cross-entropy

In [63]:
# Value class starter code, with many functions taken out
from math import exp, log

class Value:

  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"

  def __add__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')

    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward

    return out

  # ------

  def exp(self):
    out = Value(exp(self.data), (self, ), 'exp')

    def _backward():
      self.grad += out.data * out.grad
    out._backward = _backward

    return out
  
  def __sub__(self, other):
    return self + (-other)
  
  def __rsub__(self, other):
    return -self + other
  
  def __radd__(self, other):
    return self + other

  def __pow__(self, other):
    out = Value(self.data**other, (self, ), f'**{other}')

    def _backward():
      self.grad += other * (self.data ** (other - 1)) * out.grad
    out._backward = _backward

    return out

  def __truediv__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data / other.data, (self, other), '/')
  
    def _backward():
      self.grad += out.grad / other.data
      other.grad += -self.data * out.grad / (other.data**2)
      # self.grad += 1.0 / other.data * out.grad
      # other.grad += -self.data / other.data**2 * out.grad
    out._backward = _backward

    return out
  
  def __rtruediv__(self, other):
    return Value(other) / self
  
  def log(self):
    out = Value(log(self.data), (self, ), 'log')

    def _backward():
      self.grad += out.grad / self.data
    out._backward = _backward

    return out
  
  def __neg__(self):
    return self * -1
  
  def __rmul__(self, other):
    return self * other

  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, (self, other), '*')

    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward

    return out
  
  def max(self, other): # max(Value, int)
    out = Value(max(self.data, other), (self, ), 'max')

    def _backward():
      self.grad += out.grad if (self.data > other) else 0
    out._backward = _backward

    return out

  # ------

  def backward(self):
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)

    self.grad = 1.0
    for node in reversed(topo):
      node._backward() 

In [64]:
# this is the softmax function
# https://en.wikipedia.org/wiki/Softmax_function
def softmax(logits):
  counts = [logit.exp() for logit in logits]
  denominator = sum(counts)
  out = [c / denominator for c in counts]
  return out

# this is the negative log likelihood loss function, pervasive in classification
logits = [Value(0.0), Value(3.0), Value(-2.0), Value(1.0)]
probs = softmax(logits)
loss = -probs[3].log() # dim 3 acts as the label for this input example
loss.backward()
print(loss.data)

ans = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]
for dim in range(4):
  ok = 'OK' if abs(logits[dim].grad - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits[dim].grad}")


2.1755153626167147
OK for dim 0: expected 0.041772570515350445, yours returns 0.04177257051535045
OK for dim 1: expected 0.8390245074625319, yours returns 0.839024507462532
OK for dim 2: expected 0.005653302662216329, yours returns 0.00565330266221633
OK for dim 3: expected -0.8864503806400986, yours returns -0.8864503806400988


In [65]:
#this is relu
#https://en.wikipedia.org/wiki/Rectifier_(neural_networks)
def relu(inp):
  result = []
  for v in inp:
    result.append(v.max(0))
  return result

#this is sigmoid, feel free to implement by yourself
#https://en.wikipedia.org/wiki/Sigmoid_function
def sigmoid(inp):
  result = []
  for v in inp:
    result.append(1 / (1 + (-v).exp()))
  return result

In [66]:
#should do something like that: https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html
#with sum reduction
def cross_entropy(inp, target):
  probs = sigmoid(inp)
  vals = [-(t*p.log() + (1-t)*(1-p).log()) for p,t in zip(probs, target)]
  total_loss = sum(vals)
  return total_loss


#Section 3: Verification

All implemented functions should be verified by their PyTorch analogs.
Use same inputs to check gradients!

In [67]:
# verify the gradient using the torch library
# torch should give you the exact same gradient with same inputs
import torch

#example
torch_value1 = torch.Tensor([6.0]).double()
torch_value1.requires_grad = True

torch_value2 = torch.Tensor([7.0]).double()
torch_value2.requires_grad = True

torch_sum = torch_value1 + torch_value2
torch_sum.backward()

value_value1 = Value(6.0)
value_value2 = Value(7.0)

value_sum = value_value1 + value_value2

value_sum.backward()

assert torch_value1.grad.item() == value_value1.grad
assert torch_value2.grad.item() == value_value2.grad

print('OK')

OK


In [68]:
#Check RELU

torch_value = torch.Tensor([3.0, -1.0, 2.0]).double()
torch_value.requires_grad = True

torch_relu = torch.nn.functional.relu(torch_value)
torch_sum = sum(torch_relu)
torch_sum.backward()

my_value = [Value(3.0), Value(-1.0), Value(2.0)]

my_relu = relu(my_value)
my_sum = sum(my_relu)
my_sum.backward()

print(f"grad {torch_value.grad.tolist()}")
assert torch_value.grad.tolist() == [val.grad for val in my_value]

print('RELU: OK')

#Check Softmax

torch_value = torch.Tensor([3.0, -1.0, 2.0]).double()
torch_value.requires_grad = True

torch_softmax = torch.nn.functional.softmax(torch_value, dim=-1)
loss = -torch_softmax[2].log() # dim 2 acts as the label for this input example
loss.backward()

my_value = [Value(3.0), Value(-1.0), Value(2.0)]

my_softmax = softmax(my_value)
loss = -my_softmax[2].log()
loss.backward()

print(f"grad {torch_value.grad.tolist()}")
assert all([True if abs(tv - v.grad) < 1e-7 else False for tv, v in zip(torch_value.grad.tolist(), my_value)])

print('SOFTMAX: OK')

#Check Sigmoid

torch_value = torch.Tensor([3.0, -1.0, 2.0]).double()
torch_value.requires_grad = True

torch_sigmoid = torch.nn.functional.sigmoid(torch_value)
torch_sum = sum(torch_sigmoid)
torch_sum.backward()

my_value = [Value(3.0), Value(-1.0), Value(2.0)]

my_sigmoid = sigmoid(my_value)
my_sum = sum(my_sigmoid)
my_sum.backward()

print(f"grad {torch_value.grad.tolist()}")
assert all([True if abs(tv - v.grad) < 1e-7 else False for tv, v in zip(torch_value.grad.tolist(), my_value)])

print('SIGMOID: OK')

#Check Cross-Entropy

torch_value = torch.Tensor([3.0, -1.0, 2.0]).double()
torch_value.requires_grad = True

torch_loss = torch.nn.functional.binary_cross_entropy_with_logits(torch_value, torch.Tensor([0.0, 0.0, 1.0]).double(), reduction='sum')
torch_loss.backward()

my_value = [Value(3.0), Value(-1.0), Value(2.0)]

my_loss = cross_entropy(my_value, [Value(0.0), Value(0.0), Value(1.0)])
my_loss.backward()

print(f"grad {torch_value.grad.tolist()}")
assert all([True if abs(tv - v.grad) < 1e-7 else False for tv, v in zip(torch_value.grad.tolist(), my_value)])

print('CROSS_ENTROPY: OK')

grad [1.0, 0.0, 1.0]
RELU: OK
grad [0.7213991842739687, 0.013212886953789414, -0.7346120712277581]
SOFTMAX: OK
grad [0.045176659730912, 0.19661193324148185, 0.10499358540350662]
SIGMOID: OK
grad [0.9525741268224334, 0.2689414213699951, -0.11920292202211769]
CROSS_ENTROPY: OK


#Section 4: Debug code (2 pts)

This code is implemented with some errors.
Your task is to find and fix all of them.

Please write a short summary what you've fixed (ex: line 100 - typo, line 42 - invalid argument)


In [69]:
import numpy as np

def mse_loss(predictions, targets):
    """
    Returns the Mean Squared Error Loss given the
    predictions and targets

    Args :
    predictions (np. ndarray ): Model predictions
    targets (np. ndarray ): True outputs

    Returns :
    Mean squared error loss between predictions and targets
    """
    return np.mean((predictions.reshape(-1) - targets.reshape(-1))**2)  # fix


def dropout (x, p =0.1):
    """
    Applies dropout on the input x with a drop
    probability of p

    Args :
    x (np. ndarray ): 2D array input
    p ( float ): dropout probability )

    Returns :
    Array with values dropped out
    """
    ind = np.random.choice(x.shape[1] * x.shape[0], replace=False, size=int(x.shape[1] * x.shape[0] * p))
    x[np.unravel_index(ind, x.shape)] = 0 # fix
    return x / (1 - p) # fix

def get_grads(loss, w1, b1, w2, b2):
    """
    This function takes the loss and returns the gradients
    for the weights and biases
    YOU MAY ASSUME THIS FUNCTION HAS NO ERRORS
    """
    return dw1, db1, dw2, db2


def sample_batches(data, batchsize):
    """
    This function samples of batches of size `batchsize `
    from the training data .
    YOU MAY ASSUME THIS FUNCTION HAS NO ERRORS
    """
    return x, y


class Adam:
    """
    The class for the Adam optimizer that
    accepts the parameters and updates them .
    YOU MAY ASSUME THIS CLASS AND ITS METHODS HAVE NO ERRORS
    """
    def __init__ (self, w1, b1, w2, b2):
        pass

    def update(self):
        """
        Updates the params according to the
        Adam update rule
        """
        pass


class MLP:
    """
    MLP Model to perform regression
    """
    def __init__(self):
        super().__init__() # super?
        self.w1 = np.random.randn(10, 20)
        self.b1 = np.zeros(20) # fix
        self.w2 = np.random.randn(20, 1)
        self.b2 = np.zeros(1) # fix
        self.optimizer = Adam(w1, b1, w2, b2)

    def forward(self, x):
        """
        Forward pass for the model

        Args :
        x (np.ndarray ): Input of shape batchsize x 10

        Returns :
        out (np.ndarray ): Output of shape batchsize x 1
        """
        x = x.dot(self.w1) + self.b1 # fix
        x = dropout(x)
        x = x.dot(self.w2) + self.b2 # fix
        return x


    def train(self, training_data, test_data):
        """
        This method trains the neural network and outputs
        predictions for the test_data

        # Args :
        # training_data (np.ndarray ):
        # Training data containing (x, y) pairs
        # x is 10- dimensional and y is 1- dimensional
        # test_data (np.ndarray ): 100 test points of shape
        # (100 , 10)

        # Returns :
        # predictions (np.ndarray ): The predictions for
        # the 100 test points .
        # Final shape is (100 ,1)
        # """
        batchsize = 32; num_epochs = 1000
        for _ in range(num_epochs): # fix
            for x, y in sample_batches(training_data, batchsize):
                # Shape of x is (32 , 10) and y is (32 , 1)
                out = self.forward(x)
                loss = mse_loss(out, y) # fix
                dw1, db1, dw2, db2 = get_grads(loss, self.w1, self.b1, self.w2, self.b2)
            self.optimizer.update() # zero_grad?

        # Assume test_data is of shape (100 , 10)
        predictions = self.forward(test_data)
        return predictions # fix




# -------------------------------------------------
#
# fixed a common tab error before the docstring
# 
# line 15 - mean instead of 0.5
#
# line 31 - typo
#
# line 32 - wrong scaling
#
# line 74 - no super class    
#    
# line 76,78 - size fix
#
# line 91,93 - typo and matmul fix
#    
# line 115 - num_epochs not defined
#
# line 119 - out instead of x
#
# line 121 - you need to add self.optimizer.zero_grad() after update if it is not meant to be done inside update
#
# line 125 - typo
#
# -------------------------------------------------